# Make Analysis Data for R

R에서 사용할 데이터 생성

* t-test
* survival

## Load Data

In [ ]:
# 파일 불러오기
matching_list = list(np.load('New_Data/mathing_list.npy', allow_pickle=True))
df = pd.read_csv('New_Data/TCGA_GTEX_matched_label.csv')
time = pd.read_csv('New_Data/TCGA_survival_data', sep='\t')

In [ ]:
# 최대추적기간: 11252일
time.describe()

In [ ]:
# 결측값 확인
time.isnull().sum()

## Make Data

In [ ]:
# 암종별 mathing 파일 생성 후 내보내기
for match in matching_list:
    # 추출
    data = df[df['TCGA_GTEX_main_category'].isin(match)]
    data.sort_values(by='sample', inplace=True)
    data.reset_index(inplace=True, drop=True)

    # 0:GTEX, 1:TCGA
    data['label'] = 0
    data.loc[data['TCGA_GTEX_main_category'].str.contains('TCGA'), 'label'] = 1
    data.drop(columns=['TCGA_GTEX_main_category'], inplace=True)

    # GTEX_TCGA 파일 내보내기 (t-test)
    data.to_csv('New_Data/t_test/TCGA_GTEX_'+match[0].split(' ')[1]+'_t_test.csv', index=False)
    print(match[0].split(' ')[1]+'_t_test.csv:', data.shape)
    
    # OS 결합
    data_m = pd.merge(data, time, how='inner', on='sample')
    df_col = time.columns.tolist() + data_m.columns[2:-3].tolist()
    data_m_ = data_m[df_col]

    # TCGA 파일 내보내기 (survival)
    data_m_.to_csv('New_Data/survival/TCGA_'+match[0].split(' ')[1]+'_survival.csv', index=False)
    print(match[0].split(' ')[1]+'_survival.csv:', data_m.shape)